# Taller 2
# [Introducción a los sistemas inteligentes 2018-2](https://github.com/fagonzalezo/iis-2018-2/blob/master/taller1.ipynb)
**Fecha límite de entrega**: Miércoloe 24 de Octubre antes de la medianoche (ver instrucciones de envío al final)

Integrantes del grupo (máximo 3):

* Nombre_1 ID_1
* Nombre_2 ID_2
* Nombre_3 ID_3



In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pylab as pl
from sklearn.datasets import make_circles
from matplotlib.colors import Normalize


# Función para visualizar un conjunto de datos en 2D
def plot_data(X, y):
    y_unique = np.unique(y)
    colors = pl.cm.rainbow(np.linspace(0.0, 1.0, y_unique.size))
    for this_y, color in zip(y_unique, colors):
        this_X = X[y == this_y]
        pl.scatter(this_X[:, 0], this_X[:, 1],  c=color,
                    alpha=0.5, edgecolor='k',
                    label="Class %s" % this_y)
    pl.legend(loc="best")
    pl.title("Data")
    
# Función para visualizar de la superficie de decisión de un clasificador
def plot_decision_region(X, pred_fun):
    min_x = np.min(X[:, 0])
    max_x = np.max(X[:, 0])
    min_y = np.min(X[:, 1])
    max_y = np.max(X[:, 1])
    min_x = min_x - (max_x - min_x) * 0.05
    max_x = max_x + (max_x - min_x) * 0.05
    min_y = min_y - (max_y - min_y) * 0.05
    max_y = max_y + (max_y - min_y) * 0.05
    x_vals = np.linspace(min_x, max_x, 100)
    y_vals = np.linspace(min_y, max_y, 100)
    XX, YY = np.meshgrid(x_vals, y_vals)
    grid_r, grid_c = XX.shape
    ZZ = np.zeros((grid_r, grid_c))
    for i in range(grid_r):
        for j in range(grid_c):
            ZZ[i, j] = pred_fun(XX[i, j], YY[i, j])
    pl.contourf(XX, YY, ZZ, 100, cmap = pl.cm.coolwarm, vmin= -1, vmax=2)
    pl.colorbar()
    pl.xlabel("x")
    pl.ylabel("y")
    
class MidpointNormalize(Normalize):

    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))
    
def gen_pred_fun(clf):
    def pred_fun(x1, x2):
        x = np.array([[x1, x2]])
        return clf.predict(x)[0]
    return pred_fun

def list_cm(cm,classes):     #función para generar de una forma más visual la matriz de confusión
    if len(cm)==2:
      cm.astype(int)
      row_0 =['','Valor','Verdadero']
      row_1 =['-',classes[0],classes[1]]
      row_2 =[classes[0],cm[0,0],cm[1,0]]
      row_3 =[classes[1],cm[0,1],cm[1,1]]
      table = zip(row_0,row_1, row_2, row_3)
      headers = ['', '', 'Valor', 'Predicho']  
      return print(tabulate(table, headers=headers, floatfmt=".0f"))
    else:
      cm.astype(int)
      row_0 =['','Valor','Verdadero','']
      row_1 =['-',np.int(classes[0]),classes[1],classes[2]]
      row_2 =[classes[0],cm[0,0],cm[1,0],cm[2,0]]
      row_3 =[classes[1],cm[0,1],cm[1,1],cm[2,1]]
      row_4 =[classes[2],cm[0,2],cm[1,2],cm[2,2]]
      table = zip(row_0,row_1, row_2, row_3, row_4)
      headers = ['', '', 'Valor', 'Predicho', '']  
      return print(tabulate(table, headers=headers, floatfmt=".0f")) 

# 1. Clasificación de dígitos

Carguemos el conjunto de datos `MNIST`

In [0]:
from sklearn.datasets import fetch_mldata

##Vector de 784 valores
## Descarga la base MNIST (mnist-original.mat) en la carpeta 'data'
digits = fetch_mldata('MNIST original', data_home='data')

Mostramos un elemento en el conjunto:

In [0]:
idx = 9000
print('Valor dígito: {}'.format(digits.target[idx]))

plt.figure(1, figsize=(3, 3))
plt.grid(False)
plt.imshow(digits.data[idx].reshape((28,28)), cmap=plt.cm.gray_r, interpolation='nearest')
plt.show()

In [0]:
digits.data[idx].reshape((28,28))

Revisamos algunas propiedades del conjunto de datos:

In [0]:
print('Número de instancias: {}'.format(digits.data.shape[0]))
print('Número de clases: {}'.format(len(np.unique(digits.target))))
print('Clases a predecir: {}'.format(np.unique(digits.target)))

Cada instancia del conjunto de datos está representada como una matriz de tamaño $28\times 28$. Cada matriz representa una imagen en escala de grises, que contiene un dígito escrito a mano (de 0 a 9). El valor de cada elemento en la matriz representa la intensidad del color en el respectivo pixel. Verifiquemos los valores mínimos y máximos de intensidad en todas las imágenes:

In [0]:
print('Mínimo: {}'.format(np.min(digits.data)))
print('Máximo: {}'.format(np.max(digits.data)))

Esto nos indica que la intensidad de cada pixel varía entre $0$ y $255$. Verifiquemos la distribución de las clases del conjunto de datos:

In [0]:
y = np.array([int(x) for x in digits.target])

In [0]:
for label, freq in zip(np.unique(y), np.bincount(y)):
    print('Etiqueta {}: {}'.format(label, freq))

## 1.1 Escale los datos de tal forma que tengan media cero, varianza uno. 

Puede usar la función [`StandardScaler`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) 


## 1.2  Seleccione imágenes correspondientes a dos dígitos, por ejemplo, $0$ y $9$, o, $2$ y $4$.

## 1.3  Divida el conjunto de datos en **entrenamiento**, **validación** y **prueba**. 
Primero separe **prueba** usando el 30% de los datos originales, luego use el otro 70% y particionelo de tal forma que entrenamiento tenga el $80\%$ y validación el $20\%$. Recuerde que las tres particiones deben ser estratificadas.

## 1.4 Entrene un clasificador `LinearSVC`

explore el parámetro de regularización ($C=[2^{-15}, 2^{-9},2^{-8},\dots,2^{5}]$) sobre entrenamiento y validación:
* Grafique la curva de complejidad entre entrenamiento y validación. **Tip**: Use `pl.xscale("log", basex=2)` para poder visualizar la curva en escala logarítmica.
* Escoja el parámetro con el mejor compromiso entre error de validación y entrenamiento.


## 1.5 Evalue el modelo

* Una el conjunto de entrenamiento con el de validación y entrene un modelo usando el mejor parámetro.
* Reporte en el conjunto de **prueba** escalado: error, accuracy, precisión por clase, recall por clase, f_1 score por clase y la matriz de confusión.


## 1.6 Entrene un clasificador `RandomForest`

Explore el parámetro de complejidad ($n-estimators=[2^{1}, 2^{2},\dots,2^{11}]$) sobre entrenamiento y validación:
* Grafique la curva de complejidad entre entrenamiento y validación. **Tip**: Use `pl.xscale("log", basex=2)` para poder visualizar la curva en escala logarítmica.
* Escoja el parámetro con el mejor compromiso entre error de validación y entrenamiento.

## 1.7 Evalue el modelo

* Una el conjunto de entrenamiento con el de validación y entrene un modelo usando el mejor parámetro.
* Reporte en el conjunto de **prueba** escalado: error, accuracy, precisión por clase, recall por clase, f_1 score por clase y la matriz de confusión.


## 1.8 Visualización de la importancia de características

* Use el conjunto original con las 10 clases sin escalar.
* Genere una partición 70%-30% para entrenamiento y prueba.
* Entrene un modelo RandomForest con los siguientes parámetros:
    * `n_estimators`: 100
    * `random_state`: 42
* Reporte el desempeño del modelo en el conjunto de **prueba**: error, accuracy, precisión por clase, recall por clase, f_1 score por clase y la matriz de confusión.
* Use el modelo entrenado y extraiga las características más importantes. Use la siguiente pieza de código para visualizar los pixeles más importantes:

```python
importances = random_forest.feature_importances_
importances = importances.reshape((28,28))

plt.matshow(importances, cmap=plt.cm.hot)
plt.title("Importancias por pixel")
plt.show()
```

## 1.9 Entrene un modelo con todas las clases 
* Separe el conjunto de datos en prueba, validación y entrenamiento.
* Pruebe diferentes métodos de clasificación (al menos 3)
* Para cada método de clasificación  explore los mejores parámetros de complejidad usando los subconjuntos de validación y entrenamiento. Una validación y entrenamiento y entrene un modelo con los mejores parámetros.
* Pruebe los modelos obetenidos con los diferentes métodos en el conjunto de prueba. Evalue diferentes medidas de rendimiento y haga una comparación. Analice y discuta los resultados.

# 2. Clasificación de escenas

Vamos a usar el conjunto de datos [15-scene](https://figshare.com/articles/15-Scene_Image_Dataset/7007177), el cual contiene imágenes de 15 escenarios como suburbios, zonas industirales, habitaciones, entre otros. Vamos a usar una estrategia análoga a la descrita en este tutorial [ IMAGE CLASSIFICATION WITH SCIKIT-LEARN](https://kapernikov.com/tutorial-image-classification-with-scikit-learn/)


## 2.1 Cree el conjunto de datos

*  Descargue las imágenes del sitio web [15-scene](https://figshare.com/articles/15-Scene_Image_Dataset/7007177). 
*  Lea las imágenes a memoria y cree una estructura análoga al diccionario `data` creado en el tutorial (esta estructura es cargada desde un `pkl` de manera que no se describe como se creo a partir de las imágenes). Se sugiere utilizar las funciones de [Matplotlib](https://matplotlib.org/) para la carga de imágenes descritas [aquí](https://matplotlib.org/users/image_tutorial.html).
* Visualice un par de imágenes para verificar que se hayan cargado exitosamente el conjunto de datos.
* ¿Que *shape* tiene `data['data']`?
* ¿Que *shape* tiene `data['label']`?
* Haga un *split* de las imágenes en *train* y *test* de manera estratificada.
* Dibuje un *bar plot* similar al mostrado en el tutorial `'relative amount of photos per type'`



## 2.2 Extraiga las características 

Siguiendo el tutorial vamos a extraer las características de las imágenes usando HOG:
* Aplique los *transformers* descritos en el tutorial
* ¿Que *shape* tiene `X_train_prepared`?

## 2.3 Entrene diferentes clasificadores

Entrene diferentes tipos de clasificadores (logistic regression, random forest, SVM, etc.):
* Explore los valores óptimos de los hiperparámetros usando validación cruzada y *grid search*
* Grafique los resultados de la exploración
* Escoja los valorés apropiados de los hiperparámetros para cada tipo de clasificador

## 2.4 Evalue los clasificadores con las imágenes de prueba

* Aplique las transformaciones del caso al conjunto de prueba.
* Para cada uno de los clasificadores muestre la matriz de confusión de prueba
* Calcule diferentes métricas de desempeño para los diferentes clasificadores
* Analice y discuta los resultados en detalle

**Instrucciones de envío:**

Este notebook debe enviarse a través del siguiente [File Request](https://www.dropbox.com/request/gWOZlYV5R7LvaIGF0Qdw)
antes de la medianoche de la fecha límite. El archivo debe nombrarse como 
`iis-taller2-unalusername1-unalusername2-unalusername3.ipynb`, donde `unalusername` es el nombre de usuario asignado por la universidad (incluya los nombres de usuario de todos los miembros del grupo).